### First of all.I want to say ı m not a professional but ı believe that imagination is most important thing in this world.İf we can imagine we can do.Last word is by Einstein.

### -The true indicator of intelligence is imagination.Don't allow to lose it. -Albert Einstein-


In [ ]:
#import library
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

**WE WİLL DO THESE İN ORDER**

1-Exploratory Data Analysis (EDA)

2-Prepare Data to Predict

3-Using Machine to predict Heart Failure.

In [ ]:
#import data
data = pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

 # **1-EXPLORATORY DATA ANALYSİS**

**Let's look data columns if they have any null value?**

In [ ]:
data.info()

**No.Data hasn't any NaN value.So we are lucky.we won't do anything for null values.Let's look at some value of them.So we can recognize them better.**

In [ ]:
data.head(20)

This columns' values are heart failure records values.Then if we know what is the normal values we can understand who can't be heart failure better.So let's find out.





In [ ]:
data.describe()

In [ ]:
data.drop("time",axis = 1,inplace = True)

İf we look minimum values in each columns.We can understand what is the normal values,For example 'age' column's min values is 40.So we can understand minimum heart failure case happened in 40 age.İf person age lower than 40.Then he won't be heart failure most probably.

Well.Then if our person bigger than 40 ? How can understand it will be heart failure.Then we must look at other values for understand it.İf we know their normal values again we can understand heart failure before it happened.Let's talking about their normal values.

### Normal Values:

* Creatinine_phosphokinase(CPK Enzyme) : 10-120
* Ejection Fraction : 55-70
* Platelets : 150000 - 450000
* Serum Creatinine : 0.50-1.35
* Serum Sodium : 135-145

### Boolean Values:

* 0 : Positive Value
* 1 : Negative Value

### Okay.Let's make a heatmap to see columns correlations.

In [ ]:
import seaborn as sns

f, ax = plt.subplots(figsize=(12,12))
ax = sns.heatmap(data.corr(), annot = True,cmap = "YlGnBu",vmin = 0,vmax = 1)


### As we can see.Data columns has so low correlations(positive or negative) with each other.So columns has own values.Their independent variables.Just a few columns has correlations with each other.

### This Columns :

* Age,Death Event(we will examine it)
* Serum Creatinine,Death Event(we will examine it)
* Sex,Smoking(İt is not important correlation to predict,so we won't look at that)

In [ ]:
#age,death_event

bigger_than_sixty_age = data[data.age>=60][["age","DEATH_EVENT"]]
death_rate = (bigger_than_sixty_age.DEATH_EVENT.value_counts()[1]/bigger_than_sixty_age.DEATH_EVENT.value_counts()[0])*100

print("Bigger than sixty age people count :",len(bigger_than_sixty_age))
print("Death Rate in bigger than sixty ages:",death_rate)

lower_than_sixty_age = data[data.age<60][["age","DEATH_EVENT"]]
death_rate2 = (lower_than_sixty_age.DEATH_EVENT.value_counts()[1]/lower_than_sixty_age.DEATH_EVENT.value_counts()[0])*100
print("****************************************************")
print("Lower than sixty age people count :",len(lower_than_sixty_age))
print("Death Rate in lower than sixty ages:",death_rate2)

### **We are seeing if person have old,%61 probability can die,if person not so much old also can die %31 probability.**

In [ ]:
#serum_creatinine,death_event

normal_serum_creatinines = data[data.serum_creatinine<=1.35][["serum_creatinine","DEATH_EVENT"]]
anormal_serum_creatinines = data[data.serum_creatinine>1.35][["serum_creatinine","DEATH_EVENT"]]


death_rate_of_nsc = (normal_serum_creatinines.DEATH_EVENT.value_counts()[1]/normal_serum_creatinines.DEATH_EVENT.value_counts()[0])*100
death_rate_of_ansc = (anormal_serum_creatinines.DEATH_EVENT.value_counts()[1]/normal_serum_creatinines.DEATH_EVENT.value_counts()[0])*100

print("Normal Serum Creatinine People Count : ",len(normal_serum_creatinines))
print("Normal Serum Creatinine Death Rate : %",death_rate_of_nsc)
print("****************************************************")
print("Anormal Serum Creatinine People Count :",len(anormal_serum_creatinines))
print("Anormal Serum Creatinine Death Rate : %",death_rate_of_ansc)

### As we can see my hypothesis is True because:

* normal serum creatinine People Count big and it's death rate just %28 
* anormal serum creatinine People Count lower 3 times than other but it's also death rate %27

You can ask patient has normal sc value but can die,how?.Because if patient has so anormal values in other columns.Also can die. 

Then what will we do? 

### We will find how many patient died.And why?

* How many anormal value can kill a patient we will find this and then we can see if it started to be dangerous and we can predict before it happened.   


For example : (Anormal value : 5,patient died.We can see while anormal value : (2-3),we can understand patient in danger.So if we take him intensive care,actually can be good.)
We will design a measurement so it will follow that values and if anormal_values count bigger than 2-3 it will warn us.So we can predict before heart failure happened. 

# 2 - PREPARE TO DATA

In [ ]:
died_patient = data[data.DEATH_EVENT == 1]
died_patient.drop("sex",axis = 1,inplace = True)
died_patient

### Now.İf we find how many normal values in the died patients,we can actually find how many anormal value can kill a patient.You got it.Let's find them.

In [ ]:
died_patient[(died_patient.creatinine_phosphokinase<120) & 
            (died_patient.diabetes == 0) &
            ((died_patient.ejection_fraction)<70) & ((died_patient.ejection_fraction)>55)&
            (died_patient.anaemia == 0) &
            ((died_patient.platelets) > 150000) & ((died_patient.platelets) < 450000) &
            (died_patient.high_blood_pressure == 0) &
            ((died_patient.serum_creatinine) > 0.50) & ((died_patient.serum_creatinine) < 1.35) &
            ((died_patient.serum_sodium) > 135) & ((died_patient.serum_sodium) < 145)]

### Wait.We couldn't find any died patient when all of columns are normal values.But how can we doing this then.Maybe we must try that one by one.İt will be slow but it will be healthly. 

In [ ]:
#I will drop 'smoking' in here because it's not directly affective to heart failure.

died_patient.drop("smoking",axis = 1 ,inplace = True)
died_patient.head()

In [ ]:
count_ = len(died_patient[(died_patient.anaemia == 0)])
print(count_)
print("******************")
died_patient[(died_patient.anaemia == 0)]

### As we can see if just anaemia is normal other 7 columns can kill a patient(50/96 people dead).Let's keep going.I guess we will find more.****

In [ ]:
count_ = len(died_patient[(died_patient.anaemia == 0) & (died_patient.diabetes == 0)])
print(count_)
print("******************")
died_patient[(died_patient.anaemia == 0) & (died_patient.diabetes == 0)]

### As we can see.İf 2 columns normal other columns can kill 28 patients.(28/96)

In [ ]:
count_ = len(died_patient[(died_patient.anaemia == 0) & (died_patient.diabetes == 0) & (died_patient.high_blood_pressure == 0)])
print(count_)
print("******************")
died_patient[(died_patient.anaemia == 0) & (died_patient.diabetes == 0) & (died_patient.high_blood_pressure == 0)]

### Still 19 people can died.Keep going.

In [ ]:
count_ = len(died_patient[(died_patient.anaemia == 0) & (died_patient.diabetes == 0) & (died_patient.high_blood_pressure == 0) & 
            (died_patient.creatinine_phosphokinase > 10) & (died_patient.creatinine_phosphokinase <150)])
print(count_)
print("******************")
died_patient[(died_patient.anaemia == 0) & (died_patient.diabetes == 0) & (died_patient.high_blood_pressure == 0) & 
            (died_patient.creatinine_phosphokinase > 10) & (died_patient.creatinine_phosphokinase <150)]

### I guess we almost arrived.We will find it minimum killer columns count.

In [ ]:
count_ = len(died_patient[(died_patient.anaemia == 0) & (died_patient.diabetes == 0) & (died_patient.high_blood_pressure == 0) & 
            (died_patient.creatinine_phosphokinase > 10) & (died_patient.creatinine_phosphokinase <150) & 
            (died_patient.ejection_fraction > 55) & (died_patient.ejection_fraction < 70)])
print(count_)
print("******************")
died_patient[(died_patient.anaemia == 0) & (died_patient.diabetes == 0) & (died_patient.high_blood_pressure == 0) & 
            (died_patient.creatinine_phosphokinase > 10) & (died_patient.creatinine_phosphokinase <150) & 
            (died_patient.ejection_fraction > 55) & (died_patient.ejection_fraction < 70)]


### Yes.Finally we found.Look at that.5 columns are normal but anyway patient died because 3 columns are anormal.

Then what we can say about that.

* Minimum 3 columns enough to kill a patient.Then if we can design a measurement when patient's values arrive 3 count,measurement will warn to us and we will understand our patient in dangerous.Okay,can we design a measurement.Yeah,maybe.Let's look.

In [ ]:
died_patient.head(10)

### Let's design our measurement

# USİNG MACHİNE TO PREDİCT HEART FAİLURE

In [ ]:
#Machine Design

#Value creating
def create_random_values():
    creatinine_phos_ = np.random.randint(10,150)
    anaemia_ = np.random.randint(0,2)
    diabetes_ = np.random.randint(0,2)
    high_blood_press_ = np.random.randint(0,2)
    ejection_fraction_ = np.random.randint(50,80)
    platelets_ = np.random.randint(100000,500000)
    serum_creatinine_ = np.random.randint(0,4)
    serum_sodium_ = np.random.randint(110,150)

    com_list = {"creat": creatinine_phos_,
                "anaem" : anaemia_,
                "diabet" : diabetes_,
                "hbp" : high_blood_press_,
                "ef" : ejection_fraction_,
                "plat" : platelets_,
                "serum_cre" : serum_creatinine_,
                "serum_sod" : serum_sodium_}
    
    
    return com_list
    

#Measure to patient case(how many columns are anormal)
def is_dangerous_patient_(com_list):
    count = 0
    
    if com_list["creat"] > 120 : 
        count += 1
        
    if com_list["anaem"] == 1:
        count += 1
    
    if com_list["diabet"] == 1:
        count += 1
        
    if com_list["hbp"] == 1:
        count += 1
        
    if com_list["ef"] < 55 or com_list["ef"] > 70:
        count += 1
        
    if com_list["plat"] < 150000 or com_list["plat"] > 450000:
        count += 1
        
    if com_list["serum_cre"] < 0.50 or com_list["serum_cre"] > 1.35:
        count +=1
        
    if com_list["serum_sod"] < 135 or com_list["serum_sod"] > 145:
        count +=1
    
    return count

In [ ]:
#50 iter_
for each in range(50):
    com_list = create_random_values()
    count = is_dangerous_patient_(com_list)
    
    if count < 3:
        print("Patient Count : {} ----> {}".format(count,"Patient is not in danger."))
        
    elif count == 3:
        print("Patient Count : {} ----> {}".format(count,"Patient is in danger.Take him in intensive care"))
        
    elif count > 3 and count < 5:
        print("Patient Count : {} ----> {}".format(count,"The patient almost dying.Doctor have to come as fast as soon."))
        
    elif count >= 5:
        print("Patient Count : {} ----> {}".format(count,"The patient died."))

### Normally.That values always moving up-down.Values can change and machine can measure it.But usually it is like that.Thank you for reading.Press the like button and support me.Bye ! 